<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/3%20Aprendizaje%20profundo%20(II)/Sesion%205/0_RNN_paso_paso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![IDAL](https://i.imgur.com/tIKXIG1.jpg)  

#**Máster en Inteligencia Artificial Avanzada y Aplicada:  IA^3**
---

# <strong><center>Construyendo una Red Neuronal Recurrente - Paso a Paso - Ejercicio</strong>
En este cuaderno, vamos a implementar una Red Neuronal Recurrente en numpy.

Las Redes Neuronales Recurrentes (RNN) son muy efectivas para el Procesamiento del Lenguaje Natural y otras tareas de secuencia porque tienen "memoria". Pueden leer entradas $x^{langle t \rangle}$ (como palabras) de una en una, y recordar alguna información/contexto a través de las activaciones de la capa oculta que se pasan de un paso de tiempo al siguiente. Esto permite a una RNN unidireccional tomar información del pasado para procesar entradas posteriores. Una RNN bidireccional puede tomar el contexto tanto del pasado como del futuro. 

**Nota**:
- El superíndice $[l]$ indica un objeto asociado a la capa $l^{th}$. 
    - Ejemplo: $a^{[4]}$ es la activación de la capa $4^{th}$. $W^{[5]}$ y $b^{[5]}$ son los parámetros de la capa de $5^{th}$.

- El superíndice $(i)$ indica un objeto asociado al ejemplo $i^{th}$. 
    - Ejemplo: $x^{(i)}$ es la entrada del ejemplo de entrenamiento $i^{th}$.

- El superíndice $\langle t \rangle$ denota un objeto en el $t^{th}$ paso de tiempo. 
    - Ejemplo: $x^{\langle t \rangle}$ es la entrada x en el paso de tiempo $t^{th}$. $x^ {(i)\langle t \rangle}$ es la entrada en el paso de tiempo $t^{th}$ del ejemplo $i$.
    
- El guión bajo $i$ denota la entrada $i^{th}$ de un vector.
    - Ejemplo: $a^{[l]}_i$ denota la entrada $i^{th}$ de las activaciones en la capa $l$.



Importación de paquetes

*****
**NOTA**: Es importante poner al alcance de Colab el script "rnn_utils.py" suministrado. Para ello debeis subirlo al directorio temporal "/content/" de vuestra sesion en ese momento
*****

In [1]:
import numpy as np
from rnn_utils import *

In [ ]:
##INCLUYO EN ESTA CELDA EL CONTENIDO DE rnn_utils. Es un archivo .py
# que incluye algunas funciones necesarias para hacer los ejercicios.
#
# NO EJECUTAR ESTA CELDA CUANDO SE CARGUE EL ARCHIVO, tal y como se
# explica en la nota de arriba

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def initialize_adam(parameters) :
    """
    Initializes v and s as two python dictionaries with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters["W" + str(l)] = Wl
                    parameters["b" + str(l)] = bl
    
    Returns: 
    v -- python dictionary that will contain the exponentially weighted average of the gradient.
                    v["dW" + str(l)] = ...
                    v["db" + str(l)] = ...
    s -- python dictionary that will contain the exponentially weighted average of the squared gradient.
                    s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...

    """
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):
    ### START CODE HERE ### (approx. 4 lines)
        v["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        v["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
        s["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        s["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
    ### END CODE HERE ###
    
    return v, s


def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    """
    Update parameters using Adam
    
    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    learning_rate -- the learning rate, scalar.
    beta1 -- Exponential decay hyperparameter for the first moment estimates 
    beta2 -- Exponential decay hyperparameter for the second moment estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """
    
    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    # Perform Adam update on all parameters
    for l in range(L):
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
        ### START CODE HERE ### (approx. 2 lines)
        v["dW" + str(l+1)] = beta1 * v["dW" + str(l+1)] + (1 - beta1) * grads["dW" + str(l+1)] 
        v["db" + str(l+1)] = beta1 * v["db" + str(l+1)] + (1 - beta1) * grads["db" + str(l+1)] 
        ### END CODE HERE ###

        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".
        ### START CODE HERE ### (approx. 2 lines)
        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] / (1 - beta1**t)
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)] / (1 - beta1**t)
        ### END CODE HERE ###

        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
        ### START CODE HERE ### (approx. 2 lines)
        s["dW" + str(l+1)] = beta2 * s["dW" + str(l+1)] + (1 - beta2) * (grads["dW" + str(l+1)] ** 2)
        s["db" + str(l+1)] = beta2 * s["db" + str(l+1)] + (1 - beta2) * (grads["db" + str(l+1)] ** 2)
        ### END CODE HERE ###

        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".
        ### START CODE HERE ### (approx. 2 lines)
        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)] / (1 - beta2 ** t)
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)] / (1 - beta2 ** t)
        ### END CODE HERE ###

        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
        ### START CODE HERE ### (approx. 2 lines)
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * v_corrected["dW" + str(l+1)] / np.sqrt(s_corrected["dW" + str(l+1)] + epsilon)
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * v_corrected["db" + str(l+1)] / np.sqrt(s_corrected["db" + str(l+1)] + epsilon)
        ### END CODE HERE ###

    return parameters, v, s

## 1 - Propagación hacia delante en una Red Neuronal Recurrente básica

Una RNN básica que implementará tiene la estructura siguiente. En este ejemplo, $T_x = T_y$. 

![RNN_basica](https://i.imgur.com/e5aY9bf.png)

<caption><center> **Figura 1**: Basic RNN model </center></caption>

Así es como se puede implementar una RNN: 

**Pasos**:
1. Implementar los cálculos necesarios para un paso de tiempo de la RNN.
2. Implementar un bucle sobre $T_x$ pasos de tiempo para procesar todas las entradas, una a la vez. 

### 1.1 - Célula RNN

Una red neuronal recurrente puede ser vista como la repetición de una sola célula. Primero vas a implementar los cálculos para un solo paso de tiempo. La siguiente figura describe las operaciones para un solo paso de tiempo de una célula RNN. 

![RNN_forward](https://i.imgur.com/gnr9GLl.png)
<caption><center> **Figura 2**: Celda RNN básica. Toma como entrada $x^{\langle t \rangle}$ (entrada actual) y $a^{\langle t - 1\rangle}$ (estado oculto anterior que contiene información del pasado), y da como resultado $a^{\langle t \rangle}$ que se da a la siguiente célula RNN y también se utiliza para predecir $y^{\langle t \rangle}$ </center></caption>.

**Ejercicio**: Implementar la célula RNN descrita en la figura (2).

**Instrucciones**:
1. Calcular el estado oculto con activación tanh: $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$.
2. Utilizando su nuevo estado oculto $a^{\langle t \rangle}$, calcular la predicción $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$. Le proporcionamos una función: `softmax`.
3. Almacenar $(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parámetros)$ en la caché
4. Devolver $a^{\langle t \rangle}$ , $y^{\langle t \rangle}$ y caché

Vectorizaremos sobre $m$ ejemplos. Así, $x^{\langle t \rangle}$ tendrá dimensión $(n_x,m)$, y $a^{\langle t \rangle}$ tendrá dimensión $(n_a,m)$. 

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implementa un único paso adelante de la célula RNN como se describe en la figura (2)

    Argumentos:
    xt -- sus datos de entrada en el paso de tiempo "t", matriz numpy de forma (n_x, m).
    a_prev -- Estado oculto en el paso de tiempo "t-1", matriz numpy de forma (n_a, m)
    parameters -- diccionario python que contiene:
                        Wax -- Matriz de pesos que multiplica la entrada, matriz numpy de forma (n_a, n_x)
                        Waa -- Matriz de pesos multiplicando el estado oculto, matriz numpy de forma (n_a, n_a)
                        Wya -- Matriz de pesos que relaciona el estado oculto con la salida, matriz numpy de forma (n_y, n_a)
                        ba -- Bias, matriz numpy de forma (n_a, 1)
                        by -- Sesgo que relaciona el estado oculto con la salida, matriz numpy de forma (n_y, 1)
    Devuelve:
    a_next -- siguiente estado oculto, de forma (n_a, m)
    yt_pred -- predicción en el paso de tiempo "t", matriz numpy de forma (n_y, m)
    cache -- tupla de valores necesarios para el pase hacia atrás, contiene (a_next, a_prev, xt, parameters)
    """
    
    # Recupera parametros de "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### INICIA TU CODIGO AQUÍ ###(≈2 lines)
    
    # Calcula el próximo estado de activación según la formula dada
    a_next = np.tanh(np.dot(Wax,xt)+np.dot(Waa,a_prev)+ba)
    # Calcula la salida de la celda según la formula dada
    yt_pred = softmax(np.dot(Wya,a_next)+by)
    
    ### ACABA TU CODIGO AQUÍ ###
    
    # guarda valores que necesitaremos para la backward propagation en cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [3]:
# Ejecuta esta celda para comprobar. 
# Los resultados deben coincidir con "Salida esperada"

np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

('a_next[4] = ', array([ 0.59584544,  0.18141802,  0.61311866,  0.99808218,  0.85016201,
        0.99980978, -0.18887155,  0.99815551,  0.6531151 ,  0.82872037]))
('a_next.shape = ', (5, 10))
('yt_pred[1] =', array([0.9888161 , 0.01682021, 0.21140899, 0.36817467, 0.98988387,
       0.88945212, 0.36920224, 0.9966312 , 0.9982559 , 0.17746526]))
('yt_pred.shape = ', (2, 10))


**Salida esperada**: 

<table>
    <tr>
        <td>
            **a_next[4]**:
        </td>
        <td>
           [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
        </td>
    </tr>
        <tr>
        <td>
            **a_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **yt[1]**:
        </td>
        <td>
           [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
        </td>
    </tr>
        <tr>
        <td>
            **yt.shape**:
        </td>
        <td>
           (2, 10)
        </td>
    </tr>

</table>

### 1.2 - Paso adelante de la RNN 

Puedes ver una RNN como la repetición de la celda que acabas de construir. Si su secuencia de datos de entrada se lleva a cabo durante 10 pasos de tiempo, entonces usted va a copiar la célula RNN 10 veces. Cada célula toma como entrada el estado oculto de la célula anterior ($a^{\langle t-1 \rangle}$) y los datos de entrada del paso de tiempo actual ($x^{\langle t \rangle}$). Se emite un estado oculto ($a^{\langle t \rangle}$) y una predicción ($y^{\langle t \rangle}$) para este paso de tiempo.

![RNN](https://i.imgur.com/X88Lhk2.png)

<caption><center> **Figura 3**: RNN básica. La secuencia de entrada $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ se lleva a cabo durante $T_x$ pasos de tiempo. Las salidas de la red $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>



**Ejercicio**: Codificar la propagación hacia delante de la RNN descrita en la figura (3).

**Instrucciones**:
1. Crear un vector de ceros ($a$) que almacenará todos los estados ocultos computados por la RNN.
2. Inicializar el "siguiente" estado oculto como $a_0$ (estado oculto inicial).
3. Comenzar un bucle sobre cada paso de tiempo, su índice incremental es $t$ :
    - Actualizar el "siguiente" estado oculto y la caché ejecutando `rnn_cell_forward`.
    - Almacenar el "próximo" estado oculto en $a$ ($t^{th}$ posición) 
    - Almacenar la predicción en y
    - Añadir la caché a la lista de cachés
4. Retornar $a$, $y$ y las cachés

In [14]:
def rnn_forward(x, a0, parameters):
    """
    Implementa la propagación hacia delante de la red neuronal recurrente descrita en la figura (3).

    Argumentos:
    x -- Datos de entrada para cada paso de tiempo, de forma (n_x, m, T_x).
    a0 -- Estado oculto inicial, de forma (n_a, m)
    parámetros -- diccionario de python que contiene:
                        Waa -- Matriz de pesos que multiplica el estado oculto, matriz numpy de forma (n_a, n_a)
                        Wax -- Matriz de pesos multiplicando la entrada, array numpy de forma (n_a, n_x)
                        Wya -- Matriz de pesos que relaciona el estado oculto con la salida, matriz numpy de forma (n_y, n_a)
                        ba -- Matriz numpy de forma (n_a, 1)
                        by -- Sesgo que relaciona el estado oculto con la salida, matriz numpy de forma (n_y, 1)

    Devuelve:
    a -- Estados ocultos para cada paso de tiempo, matriz numpy de forma (n_a, m, T_x)
    y_pred -- Predicciones para cada paso de tiempo, matriz numpy de forma (n_y, m, T_x)
    caches -- tupla de valores necesarios para el pase hacia atrás, contiene (lista de caches, x)
    """
    
    # Inicializa "caches" que contendrá la lista de caches
    caches = []
    
    # Recupera dimensiones de x y Wy
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### INICIA TU CODIGO AQUÍ ###
    
    # Inicializa "a" y "y" con ceros (≈2 lines)
    a = np.zeros(shape=(n_a, m, T_x))
    y_pred = np.zeros(shape=(n_y, m, T_x))
    
    # Inicializa a_next (≈1 line)
    a_next = a0
    
    # bucle sobre los time-steps
    for t in range(T_x):
        #  Actualiza próximo hidden state, calcula la prediccion, coge la cache (≈1 line)
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a, parameters)
        # Guarda el valor del "nuevo" estado oculto en a (≈1 line)
        a[:,:,t] = a_next
        # Guarda el calor de la prediccion en y (≈1 line)
        y_pred[:,:,t] = yt_pred
        # Añade "cache" a "caches" (≈1 line)
        
        
    ### ACABA TU CODIGO AQUÍ ###
    
    # guarda valores que necesitaremos para la backward propagation en cache
    caches = (caches, x)
    
    return a, y_pred, caches

In [15]:
# Ejecuta esta celda para comprobar. 
# Los resultados deben coincidir con "Salida esperada"

np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a[4][1] = ", a[4][1])
print("a.shape = ", a.shape)
print("y_pred[1][3] =", y_pred[1][3])
print("y_pred.shape = ", y_pred.shape)
print("caches[1][1][3] =", caches[1][1][3])
print("len(caches) = ", len(caches))

ValueError: ignored

**Salida esperada**:

<table>
    <tr>
        <td>
            **a[4][1]**:
        </td>
        <td>
           [-0.99999375  0.77911235 -0.99861469 -0.99833267]
        </td>
    </tr>
        <tr>
        <td>
            **a.shape**:
        </td>
        <td>
           (5, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **y[1][3]**:
        </td>
        <td>
           [ 0.79560373  0.86224861  0.11118257  0.81515947]
        </td>
    </tr>
        <tr>
        <td>
            **y.shape**:
        </td>
        <td>
           (2, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **cache[1][1][3]**:
        </td>
        <td>
           [-1.1425182  -0.34934272 -0.20889423  0.58662319]
        </td>
    </tr>
        <tr>
        <td>
            **len(cache)**:
        </td>
        <td>
           2
        </td>
    </tr>

</table>

Enhorabuena. Has construido con éxito la propagación hacia delante de una red neuronal recurrente desde cero. Esto funcionará lo suficientemente bien para algunas aplicaciones, pero sufre de problemas, uno de los principales es la fuga  de gradiente o *vanishing gradient*. Por lo tanto, funciona mejor cuando cada salida $y^{\langle t \rangle}$ se puede estimar utilizando principalmente el contexto "local" (es decir, la información de las entradas $x^{\langle t' \rangle}$ donde $t'$ no está demasiado lejos de $t$). 

A continuación vamos a estudiar su retropropagación para ir actualizando los pesos e ir mejorando los resultados. 

## 2 - Retropropagación en redes neuronales recurrentes 

En los entornos y librerías  modernos de aprendizaje profundo, sólo tienes que implementar el pase hacia adelante, y el entorno se encarga del pase hacia atrás, por lo que la mayoría de los ingenieros de aprendizaje profundo no necesitan molestarse con los detalles del pase hacia atrás. Sin embargo, si eres un experto en cálculo y quieres comprender bien los detalles del backprop en las RNN, debes trabajar en esta parte opcional del cuaderno. 

En una rede neuronal "normal" o *feedforward*  se emplea la retropropagación para calcular las derivadas con respecto al coste para actualizar los parámetros. Del mismo modo, en las redes neuronales recurrentes puedes calcular las derivadas con respecto al coste para actualizar los parámetros. Las ecuaciones de backprop son bastante complicadas y no las derivamos en la clase. Sin embargo, las presentaremos brevemente a continuación. 

### 2.1 - Retropropagación en una RNN básica

Comenzaremos calculando el pase hacia atrás para una celda RNN básica.
![RNN_backward](https://i.imgur.com/8eTNKcM.png)

<caption><center> **Figura 5**: El paso hacia atrás de la célula RNN. Al igual que en una red neuronal totalmente conectada, la derivada de la función de coste $J$ se propaga hacia atrás a través de la RNN siguiendo la regla de la cadena de cálculos. La regla de la cadena también se utiliza para calcular $(\frac{\partial J}{\partial W_{ax}},\frac{\partial J}{\partial W_{aa}},\frac{\partial J}{\partial b})$ para actualizar los parámetros $(W_{ax}, W_{aa}, b_a)$. </center></caption>

### Derivación de las funciones de un paso hacia atrás: 

Para calcular la `rnn_cell_backward` hay que calcular las siguientes ecuaciones. Es un buen ejercicio derivarlas a mano. 

La derivada de $\tanh$ es $1-\tanh(x)^2$. Puedes encontrar la demostración completa [aquí](https://www.wyzant.com/resources/lessons/math/calculus/derivative_proofs/tanx). Observa que: $ \sec(x)^2 = 1 - \tanh(x)^2$

De forma análoga para $\frac{ \partial a^{\langle t \rangle}} {\partial W_{ax}}, \frac{\partial a^{\langle t \rangle}} {\partial W_{aa}}, \frac{\partial a^{\langle t \rangle}} {\partial b}$, la derivada de $\tanh(u)$ es $(1-\tanh(u)^2)du$. 

Las dos últimas ecuaciones también siguen la misma regla y se derivan utilizando la derivada de $\tanh$. Obsérvese que el arreglo se hace de manera que coincidan las mismas dimensiones.

In [ ]:
def rnn_cell_backward(da_next, cache):
    """
    Implementa el backward pass para la célula RNN (un solo paso de tiempo).

    Argumentos:
    da_next -- Gradiente de pérdida con respecto al siguiente estado oculto
    cache -- diccionario python que contiene valores útiles (salida de rnn_cell_forward())

    Devuelve:
    gradientes -- diccionario de python que contiene:
                        dx -- Gradientes de los datos de entrada, de forma (n_x, m)
                        da_prev -- Gradientes del estado oculto anterior, de forma (n_a, m)
                        dWax -- Gradientes de los pesos de entrada a la ocultación, de forma (n_a, n_x)
                        dWaa -- Gradientes de los pesos ocultos, de forma (n_a, n_a)
                        dba -- Gradientes del vector de sesgo, de forma (n_a, 1)
    """
    ### INICIA  TU CODIGO AQUÍ ###

    # Recupera valores de cache
    (a_next, a_prev, xt, parameters) = cache
    
    # Recupera valores de parameters
    Wax = parameters['Wax']
    Waa = parameters['Waa']
    Wya = parameters['Wya']
    ba = parameters['ba']
    by = parameters['by']

    # Calcula el gradiente de tanh con respecto a a_next (≈1 line)
    dtanh = (1 - a_next ** 2) * da_next

    # Calcula el gradiente of the loss con respecto a Wax (≈2 lines)
    dxt = 
    dWax = 

    # Calcula el gradiente con respecto a Waa (≈2 lines)
    da_prev = 
    dWaa = 

    # Calcula el gradiente con respecto a b (≈1 line)
    dba = 
    
    # Guarda los gradientes en un diccionario
    gradients = { }
    
    ### ACABA TU CODIGO AQUÍ ###


    return gradients

In [ ]:
# Ejecuta esta celda para comprobar. 
# Los resultados deben coincidir con "Salida esperada"

np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Wax = np.random.randn(5,3)
Waa = np.random.randn(5,5)
Wya = np.random.randn(2,5)
b = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}

a_next, yt, cache = rnn_cell_forward(xt, a_prev, parameters)

da_next = np.random.randn(5,10)
gradients = rnn_cell_backward(da_next, cache)
print("gradients[\"dxt\"][1][2] =", gradients["dxt"][1][2])
print("gradients[\"dxt\"].shape =", gradients["dxt"].shape)
print("gradients[\"da_prev\"][2][3] =", gradients["da_prev"][2][3])
print("gradients[\"da_prev\"].shape =", gradients["da_prev"].shape)
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWax\"].shape =", gradients["dWax"].shape)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape)
print("gradients[\"dba\"][4] =", gradients["dba"][4])
print("gradients[\"dba\"].shape =", gradients["dba"].shape)

('gradients["dxt"][1][2] =', -0.4605641030588796)
('gradients["dxt"].shape =', (3, 10))
('gradients["da_prev"][2][3] =', 0.08429686538067724)
('gradients["da_prev"].shape =', (5, 10))
('gradients["dWax"][3][1] =', 0.39308187392193034)
('gradients["dWax"].shape =', (5, 3))
('gradients["dWaa"][1][2] =', -0.28483955786960663)
('gradients["dWaa"].shape =', (5, 5))
('gradients["dba"][4] =', array([0.80517166]))
('gradients["dba"].shape =', (5, 1))


**Salida esperada**:

<table>
    <tr>
        <td>
            **gradients["dxt"][1][2]** =
        </td>
        <td>
           -0.460564103059
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dxt"].shape** =
        </td>
        <td>
           (3, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"][2][3]** =
        </td>
        <td>
           0.0842968653807
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWax"][3][1]** =
        </td>
        <td>
           0.393081873922
        </td>
    </tr>
            <tr>
        <td>
            **gradients["dWax"].shape** =
        </td>
        <td>
           (5, 3)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"][1][2]** = 
        </td>
        <td>
           -0.28483955787
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"].shape** =
        </td>
        <td>
           (5, 5)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"][4]** = 
        </td>
        <td>
           [ 0.80517166]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### Paso hacia atrás a través de la RNN

El cálculo de los gradientes del coste con respecto a $a^{\langle t \rangle}$ en cada paso de tiempo $t$ es útil porque es lo que ayuda a que el gradiente se retropropague a la célula RNN anterior. Para ello, hay que iterar por todos los pasos de tiempo empezando por el final, y en cada paso, se incrementa el conjunto $db_a$, $dW_{aa}$, $dW_{ax}$ y se almacena $dx$.

**Instrucciones**:

Implementar la función `rnn_backward`. Inicializar las variables de retorno con ceros en primer lugar y luego bucle a través de todos los pasos de tiempo, mientras que llamamos a la `rnn_cell_backward` en cada paso de tiempo y actualizamos las otras variables en consecuencia.

In [ ]:
def rnn_backward(da, caches):
    """
    Implementa el backward pass para una RNN sobre una secuencia completa de datos de entrada.

    Argumentos:
    da -- gradientes ascendentes de todos los estados ocultos, de forma (n_a, m, T_x)
    caches -- tupla que contiene información del pase hacia adelante (rnn_forward)
    
    Devuelve
    gradientes -- diccionario python que contiene:
                        dx -- Gradiente con respecto a los datos de entrada, matriz numpy de forma (n_x, m, T_x)
                        da0 -- Gradiente con respecto al estado oculto inicial, matriz numpy de forma (n_a, m)
                        dWax -- Gradiente respecto a la matriz de pesos de entrada, matriz numpy de forma (n_a, n_x)
                        dWaa -- Gradiente respecto a la matriz de pesos del estado oculto, matriz numpy de shape (n_a, n_a)
                        dba -- Gradiente respecto al sesgo, de forma (n_a, 1)
    """
    ### INICIA TU CODIGO AQUÍ ###

    # Recupera valores de la primera cache (t=1) de caches (≈2 lines)
    (caches, x) = 
    (a1, a0, x1, parameters) = 
    
    # Recupera dimensions de da's y x1's  (≈2 lines)
    n_a, m, T_x = 
    n_x, m = 
    
    # Inicializa los gradientes con los tamaños correctos (≈6 lines)
    dx = 
    dWax = 
    dWaa = 
    dba = 
    da0 = 
    da_prevt = 
        
    
    # Bucle a través de todos los time steps
    for t in reversed(range(T_x)):
        # Calcula gradiente en el time step t. 
        # Escoge bien el "da_next" y el "cache" a usar en el paso de backward propagation. (≈1 line)
        gradients = 
        # Recupera derivadas de los gradientes (≈ 1 line)
        dxt, da_prevt, dWaxt, dWaat, dbat = 
        # Incrementa las derivadas globales w.r.t parameters sumandoles su derivada en el time-step t (≈4 lines)
        dx[:, :, t] = 
        dWax += 
        dWaa += 
        dba += 
        
    # Pon en da0 el gradiente del que ha sido retropropagado a través de todos los time-steps (≈1 line) 
    da0 = 

    # Guarda los gradientes en un diccionario
    gradients = {}
    
    ### ACABA TU CODIGO AQUÍ ###

    return gradients

In [ ]:
# Ejecuta esta celda para comprobar. 
# Los resultados deben coincidir con los indicados

np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Wax = np.random.randn(5,3)
Waa = np.random.randn(5,5)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}
a, y, caches = rnn_forward(x, a0, parameters)
da = np.random.randn(5, 10, 4)
gradients = rnn_backward(da, caches)

print("gradients[\"dx\"][1][2] =", gradients["dx"][1][2])
print("gradients[\"dx\"].shape =", gradients["dx"].shape)
print("gradients[\"da0\"][2][3] =", gradients["da0"][2][3])
print("gradients[\"da0\"].shape =", gradients["da0"].shape)
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWax\"].shape =", gradients["dWax"].shape)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape)
print("gradients[\"dba\"][4] =", gradients["dba"][4])
print("gradients[\"dba\"].shape =", gradients["dba"].shape)

('gradients["dx"][1][2] =', array([-2.07101689, -0.59255627,  0.02466855,  0.01483317]))
('gradients["dx"].shape =', (3, 10, 4))
('gradients["da0"][2][3] =', -0.31494237512664996)
('gradients["da0"].shape =', (5, 10))
('gradients["dWax"][3][1] =', 11.264104496527777)
('gradients["dWax"].shape =', (5, 3))
('gradients["dWaa"][1][2] =', 2.303333126579893)
('gradients["dWaa"].shape =', (5, 5))
('gradients["dba"][4] =', array([-0.74747722]))
('gradients["dba"].shape =', (5, 1))


**Salida esperada**:

<table>
    <tr>
        <td>
            **gradients["dx"][1][2]** =
        </td>
        <td>
           [-2.07101689 -0.59255627  0.02466855  0.01483317]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dx"].shape** =
        </td>
        <td>
           (3, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"][2][3]** =
        </td>
        <td>
           -0.314942375127
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
         <tr>
        <td>
            **gradients["dWax"][3][1]** =
        </td>
        <td>
           11.2641044965
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWax"].shape** =
        </td>
        <td>
           (5, 3)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"][1][2]** = 
        </td>
        <td>
           2.30333312658
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"].shape** =
        </td>
        <td>
           (5, 5)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"][4]** = 
        </td>
        <td>
           [-0.74747722]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### Enhorabuena 

Enhorabuena por haber completado esta tarea. Ahora entiendes bien cómo funcionan las redes neuronales recurrentes. 




##Referencias

*   Este ejercicio está tomado del excelente curso de Andrew Ng  "Deep Learning", accesible a través de Coursera: https://es.coursera.org/
* Doc oficial Pytorch https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
* https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

## Fin del Notebook